# Clip & Categorize SPEI

a. stein 1.18.2023

Getting SPEI setup to compare to WA USDM, following the work of `explore/usdm_spi_explore_workflow.ipynb`

In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import xarray as xr
import rioxarray
import rasterio as rio
import pandas as pd
import geopandas as gpd

import matplotlib.dates as mdates

from tqdm.autonotebook import tqdm

import sys
sys.path.append('../../')
import ndrought.wrangle as wrangle
import ndrought.compare as compare
import ndrought.plotting as ndplot

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/tmp/ipykernel_1699099/1988245056.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Load in USDM and SPEI

In [2]:
dm_path = '/pool0/home/steinadi/data/drought/drought_impact/data/drought_measures'

spei_1y = xr.open_dataset(f'{dm_path}/spei/spei1y.nc')

In [3]:
spei_1y

<xarray.Dataset>
Dimensions:  (lon: 1386, lat: 585, day: 3066, crs: 1)
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -67.14 -67.1 -67.06
  * lat      (lat) float64 49.4 49.36 49.32 49.28 ... 25.19 25.15 25.11 25.07
  * day      (day) datetime64[ns] 1980-01-05 1980-01-10 ... 2021-12-31
  * crs      (crs) uint16 3
Data variables:
    spei     (day, lat, lon) float32 ...
Attributes: (12/24)
    geospatial_bounds_crs:      EPSG:4326
    Conventions:                CF-1.6
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_lat_min:         25.066666666666666
    geospatial_lat_max:         49.40000000000000
    geospatial_lon_min:         -124.7666666333333
    ...                         ...
    last_permanent_slice:       3006
    last_early_slice:           3066
    last_provisional_slice:     3060
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

In [4]:
spei_1y.crs

<xarray.DataArray 'crs' (crs: 1)>
array([3], dtype=uint16)
Coordinates:
  * crs      (crs) uint16 3
Attributes:
    grid_mapping_name:            latitude_longitude
    longitude_of_prime_meridian:  0.0
    semi_major_axis:              6378137.0
    long_name:                    WGS 84
    inverse_flattening:           298.257223563
    GeoTransform:                 -124.7666666333333 0.041666666666666 0  49....
    spatial_ref:                  GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["...

In [5]:
spei_1y_da = spei_1y['spei'].rio.write_crs('EPSG:4326', inplace=True)

Clip SPEI to CONUS 105W

In [6]:
spei_da_105w = spei_1y_da.sel(lon=slice(-105))

In [7]:
spei_da_105w = spei_da_105w.rename({'day':'time'})

In [8]:
spei_da_105w_reproj = spei_da_105w.rio.reproject('EPSG:5070')

In [9]:
spei_da_105w_reproj

<xarray.DataArray 'spei' (time: 3066, y: 702, x: 525)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * x        (x) float64 -2.893e+06 -2.889e+06 ... -6.674e+05 -6.632e+05
  * y        (y) float64 3.251e+06 3.246e+06 3.242e+06 ... 2.717e+05 2.674e+05
  * time     (time) datetime64[ns] 1980-01-05 1980-01-10 ... 2021-12-31
    crs      int64 0
Attributes:
    units:              
    description:        Standardized Precipitation-Evapotranspiration Index
    long_name:          spei1y
    standard_name:      spei1y
    dimensions:         lon lat time
    grid_mapping:       crs
    coordinate_system:  WGS84,EPSG:4326

### Convert SPEI to USDM Categories

USDM doesn't specifically have a conversion listed for SPEI, although I bet it'd be similar to SPI. However, I'm just going to use the percentiles as follows:

D0 : 21 to 30 p    
D1 : 11 to 20 p    
D2 : 6 to 10 p    
D3 : 3 to 5 p    
D4 : 0 to 2 p

And construct the percentile limits based on the whole distribution

In [10]:
spei_vals = spei_da_105w_reproj.values.ravel()
spei_vals = spei_vals[np.isnan(spei_vals) == False]

In [11]:
for p in [30, 20, 10, 5, 2]:
    print(p, f'{np.percentile(spei_vals, p):.2f}')

30 -0.54
20 -0.90
10 -1.28
5 -1.69
2 -2.09


Okay, so pretty close, but still a bit different. Worth the difference in my opinion

In [12]:
def dm_to_usdmcat(da:xr.DataArray):
    """Categorizes drought measure based on USDM categories.

    Uses the mapping scheme presented by USDM (https://droughtmonitor.unl.edu/About/AbouttheData/DroughtClassification.aspx)
    Where Neutral is -1, D0 is 0, D1 is 1, D2, is 2, D3 is 3, and D4 is 4.

    Parameters
    ----------
    da : xr.DataArray
        Contains SPI values.
    
    Returns
    -------
    xr.DataArray
        DataArray formatted the same as da but using USDM categories.

    """

    # make sure we don't overwrite the original
    da_copy = da.copy()
    # can only do boolean indexing on the underlying array
    da_vals = da.values
    da_vals_nonnan = da_vals[np.isnan(da_vals) == False]
    # calculate percentiles
    (p30, p20, p10, p5, p2) = np.percentile(da_vals_nonnan.ravel(), [30, 20, 10, 5, 2])
    # get a copy to make sure reassignment isn't compounding
    da_origin = da_vals.copy()

    # assign neutral
    da_vals[da_origin > p30] = -1
    # assign D0
    da_vals[(da_origin <= p30)&(da_origin > p20)] = 0
    # assign D1
    da_vals[(da_origin <= p20)&(da_origin > p10)] = 1
    # assign D2
    da_vals[(da_origin <= p10)&(da_origin > p5)] = 2
    # assign D3
    da_vals[(da_origin <= p5)&(da_origin > p2)] = 3
    # assign D4
    da_vals[(da_origin <= p2)] = 4

    # put them back into the dataarray
    da_copy.loc[:,:] = da_vals

    return da_copy

In [13]:
def dm_to_usdmcat_multtime(ds:xr.Dataset):
    """Categorizes drought measure based on USDM categories for multiple times.
    
    See dm_to_usdmcat for further documentation.
    
    Parameters
    ----------
    spi_ds : xr.Dataset
        SPI at multiple time values as the coordinate 'day'.
    
    Returns
    -------
    xr.Dataset
        Drought measure categorized by dm_to_usdmcat.
    """
    
    return dm_to_usdmcat(xr.concat([ds.sel(time=t) for t in ds['time'].values], dim='time'))

In [14]:
spei_cat = dm_to_usdmcat_multtime(spei_da_105w_reproj)
spei_cat

<xarray.DataArray 'spei' (time: 3066, y: 702, x: 525)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * x        (x) float64 -2.893e+06 -2.889e+06 ... -6.674e+05 -6.632e+05
  * y        (y) float64 3.251e+06 3.246e+06 3.242e+06 ... 2.717e+05 2.674e+05
  * time     (time) datetime64[ns] 1980-01-05 1980-01-10 ... 2021-12-31
    crs      int64 0
Attributes:
    units:              
    description:        Standardized Precipitation-Evapotranspiration Index
    long_name:          spei1y
    standard_name:      spei1y
    dimensions:         lon lat time
    grid_mapping:       crs
    coordinate_system:  WGS84,EPSG:4326

Fab, that works

Let's clip all the SPEI and also save out the USDM categorized SPEI.

In [15]:
spei_intervals = ['1y', '2y', '5y', '14d', '30d', '90d', '180d', '270d']

for interval in tqdm(spei_intervals):
    spei_ds = xr.open_dataset(f'{dm_path}/spei/spei{interval}.nc')
    spei_da = spei_ds['spei'].rio.write_crs('EPSG:4326', inplace=True)
    spei_da_clip = spei_da.sel(lon=slice(-105))
    spei_da_clip = spei_da_clip.rename({'day':'time'})
    spei_da_clip = spei_da_clip.rio.reproject('EPSG:5070')
    attrs = spei_da_clip.attrs
    attrs['Clipping'] = 'This selection has been clipped to everything west of longitude 105 degrees within CONUS. EPSG:5070 was picked to preserve area for future computations.'
    del attrs['grid_mapping']
    spei_da_clip.attrs = attrs

    try:
        os.remove(f'{dm_path}/spei/CONUS_105W/spei_{interval}.nc')
    except:
        pass

    spei_da_clip.to_netcdf(f'{dm_path}/spei/CONUS_105W/spei_{interval}.nc')

    # do some gc
    spei_ds = None
    spei_da = None
    spei_da_clip = None
    

  0%|          | 0/8 [00:00<?, ?it/s]